<a href="https://colab.research.google.com/github/Satvik-Gautam/Genomic-Medicine/blob/main/Genemedicinemodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive


In [ ]:
drive.mount("/content/drive" , force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load your dataset
# Replace 'your_dataset.csv' with the actual file path
dataset_path = '/content/drive/MyDrive/Data/hack2.csv'
df = pd.read_csv(dataset_path)

# Assuming 'drug_name' is the target variable
target_column = 'drug_name'

# Extract features (X) and target variable (y)
X = df.drop(columns=[target_column])
y = df[target_column]

# Use LabelEncoder to convert string labels to numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# One-hot encode categorical features
X_encoded = pd.get_dummies(X, columns=X.select_dtypes(include='object').columns)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

# Build a simple neural network model for classification
model = Sequential()
model.add(Dense(64, input_dim=X_scaled.shape[1], activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Softmax for classification

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Train the model
history = model.fit(X_scaled, y_encoded, epochs=20, batch_size=32, validation_split=0.2)

# Print accuracy
accuracy = history.history['accuracy'][-1]
print(f"Final Accuracy: {accuracy * 100:.2f}%")

# Save the model
model.save('your_saved_model.h5')


Epoch 1/20
431/431 [==============================] - 4s 5ms/step - loss: 1.2157 - accuracy: 0.6627 - val_loss: 9.6992 - val_accuracy: 0.0302
Epoch 2/20
431/431 [==============================] - 2s 5ms/step - loss: 0.1087 - accuracy: 0.9516 - val_loss: 10.8766 - val_accuracy: 0.0305
Epoch 3/20
431/431 [==============================] - 2s 5ms/step - loss: 0.0765 - accuracy: 0.9603 - val_loss: 11.5817 - val_accuracy: 0.0305
Epoch 4/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0579 - accuracy: 0.9753 - val_loss: 12.7380 - val_accuracy: 0.0305
Epoch 5/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0353 - accuracy: 0.9875 - val_loss: 14.2241 - val_accuracy: 0.0305
Epoch 6/20
431/431 [==============================] - 2s 4ms/step - loss: 0.0148 - accuracy: 0.9962 - val_loss: 15.4611 - val_accuracy: 0.0305
Epoch 7/20
431/431 [==============================] - 1s 3ms/step - loss: 0.0084 - accuracy: 0.9979 - val_loss: 16.6924 - val_accuracy: 0.0305


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
specific_feature_name = 'ABCB1_mut'  # Change this to the desired feature name

# Load the saved model for evaluation
evaluation_model = Model(inputs=input_layer, outputs=[drug_name_output, ic50_effect_size_output, target_pathway_output])
evaluation_model.compile(optimizer='adam', loss={'drug_name': 'sparse_categorical_crossentropy', 'ic50_effect_size': 'mean_squared_error', 'target_pathway': 'sparse_categorical_crossentropy'})

# Process user input to match the input format used during training
user_input = pd.DataFrame([specific_feature_name], columns=['feature_name'])

# One-hot encode the user input with the same columns as during training
user_input_encoded = pd.get_dummies(user_input, prefix='feature_name')
user_input_encoded = user_input_encoded.reindex(columns=X_encoded.columns, fill_value=0)

user_input_scaled = scaler.transform(user_input_encoded)

# Use the model to predict drug_name, ic50_effect_size, target, and target_pathway based on the user input
predictions = evaluation_model.predict(user_input_scaled)

# Display the search results without the predicted results
search_results = df[df['feature_name'] == specific_feature_name][['feature_name'] + target_columns]
search_results_sorted = search_results.sort_values(by='ic50_effect_size', ascending=False)

print("\nSearch Results:")
print(search_results_sorted.to_markdown(index=False))


1/1 [==============================] - 0s 121ms/step

Search Results:
| feature_name   | drug_name                                 |   ic50_effect_size | target                                    | target_pathway                    |
|:---------------|:------------------------------------------|-------------------:|:------------------------------------------|:----------------------------------|
| ABCB1_mut      | Gemcitabine                               |          0.49869   | Pyrimidine antimetabolite                 | DNA replication                   |
| ABCB1_mut      | Mitoxantrone                              |          0.494241  | TOP2                                      | DNA replication                   |
| ABCB1_mut      | Bortezomib                                |          0.392577  | Proteasome                                | Protein stability and degradation |
| ABCB1_mut      | Bleomycin (50 uM)                         |          0.381678  | dsDNA break induction     